## 1. Feature Evaluation

In this file, we will be evaluating features. The procedure is as follow,

1. Subject-Dependent evaluation: build model for each subject
2. 10-CrossValidation: with 75-25 train-test ration
3. Trial-Independent: Split first then segment
4. The DEAP data set contain 63 seconds long trial, discard the first 3 seconds and segment the trial into 60 samples. (1 second long)

Plan 1
====

In Sklearn, there is a function `sklearn.model_selection.StratifiedGroupKFold`.

```
StratifiedGroupKFold is a cross-validation scheme that combines both StratifiedKFold and GroupKFold. The idea is to try to preserve the distribution of classes in each split while keeping each group within a single split. That might be useful when you have an unbalanced dataset so that using just GroupKFold might produce skewed splits.
```

There are three ideas.

1. K-fold: It is the only option.
2. Stratified: This ensure that the ratio of labels is equal for both train and test set.
3. Group: Using `Group` strategy, I can ensure the `Trial-Independent`. (each segment from the same trial has the same group)

We can not set the ratio of train-test

Plan 2
====

`sklearn.model_selection.GroupShuffleSplit`


There are two ideas.

1. ShuffleSplit: We can set the ratio of train-test
2. Group: Using `Group` strategy, I can ensure the `Trial-Independent`. (each segment from the same trial has the same group)

We can not ensure the ratio between of lebels in both train-test

In [40]:
import time
import numpy as np
from tqdm.notebook import tqdm

from components.dataset_jo import *

dataset = Dataset_subjectDependent('data')
dataset.set_segment(60)
for filename in dataset.get_file_list():
    data, labels, _ = dataset.get_data(filename, return_type='numpy')
    print(filename, labels.shape, labels.sum(axis=0)/labels.shape[0]*100)

Found: 32 files
s01 (2400, 2) [47.5 60. ]
s02 (2400, 2) [55. 60.]
s03 (2400, 2) [55. 20.]
s04 (2400, 2) [40. 40.]
s05 (2400, 2) [60.  47.5]
s06 (2400, 2) [75.  42.5]
s07 (2400, 2) [70.  62.5]
s08 (2400, 2) [55.  57.5]
s09 (2400, 2) [50. 60.]
s10 (2400, 2) [50. 55.]
s11 (2400, 2) [60.  37.5]
s12 (2400, 2) [52.5 82.5]
s13 (2400, 2) [42.5 85. ]
s14 (2400, 2) [50.  67.5]
s15 (2400, 2) [50.  47.5]
s16 (2400, 2) [37.5 50. ]
s17 (2400, 2) [55. 60.]
s18 (2400, 2) [60.  62.5]
s19 (2400, 2) [57.5 67.5]
s20 (2400, 2) [57.5 77.5]
s21 (2400, 2) [52.5 80. ]
s22 (2400, 2) [45. 60.]
s23 (2400, 2) [65. 25.]
s24 (2400, 2) [45.  82.5]
s25 (2400, 2) [47.5 72.5]
s26 (2400, 2) [65.  42.5]
s27 (2400, 2) [75.  67.5]
s28 (2400, 2) [62.5 45. ]
s29 (2400, 2) [57.5 62.5]
s30 (2400, 2) [67.5 47.5]
s31 (2400, 2) [57.5 47.5]
s32 (2400, 2) [50.  67.5]


In [2]:
# You can get the data in to way
# First way, the numpy array
data, labels, groups = dataset.get_data(name='s01', stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
print(data.shape, labels.shape, groups.shape)
print()
data, labels, groups = dataset.get_data(name='s01', stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
print(data, labels.shape, groups.shape)
print()

# Second way, as pytorch.util.Dataset
train_dataset, test_dataset = dataset.get_data_torch_dataset('s01', Dataset_subjectDependent.STIMULI_VALENCE, train_size=0.75)
print(len(train_dataset), len(test_dataset) )
# This train_dataset, and test_dataset is DataLoader ready
# train_loader = torch.utils.data.DataLoader(train_dataset, batch=8)

(2400, 32, 128) (2400, 1) (2400,)

Not setting metadata
2400 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
<EpochsArray |  2400 events (all good), 0 - 0.992188 sec, baseline off, ~75.0 MB, data loaded,
 '1': 2400> (2400, 1) (2400,)

1800 600


In [3]:
from sklearn.model_selection import GroupShuffleSplit, StratifiedGroupKFold,StratifiedShuffleSplit 
print("===== GroupShuffleSplit =====")
X,y,groups = dataset.get_data('s01', Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
print(X.shape, y.squeeze().shape, groups.shape)
cv = GroupShuffleSplit(n_splits=10, train_size=0.75, random_state=0)
for train_idxs, test_idxs in cv.split(X, y.squeeze(), groups):
    print(train_idxs.shape, test_idxs.shape, set(groups[train_idxs]).intersection(groups[test_idxs]) )
    print(sum(y[train_idxs])/len(train_idxs),sum(y[test_idxs])/len(test_idxs))

print()

print("===== StratifiedGroupKFold =====")
X,y,groups = dataset.get_data('s01', Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
print(X.shape, y.squeeze().shape, groups.shape)
cv = StratifiedGroupKFold(n_splits=10, shuffle=True, random_state=0)
for train_idxs, test_idxs in cv.split(X, y.squeeze(), groups):
    print(train_idxs.shape, test_idxs.shape, set(groups[train_idxs]).intersection(groups[test_idxs]) )
    print(sum(y[train_idxs])/len(train_idxs),sum(y[test_idxs])/len(test_idxs))

print()

print("===== StratifiedShuffleSplit =====")
X,y,groups = dataset.get_data('s01', Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
print(X.shape, y.squeeze().shape, groups.shape)
cv = StratifiedShuffleSplit(n_splits=10, train_size=0.75, random_state=0)
for train_idxs, test_idxs in cv.split(X, y.squeeze(), groups):
    print(train_idxs.shape, test_idxs.shape, set(groups[train_idxs]).intersection(groups[test_idxs]) )
    print(sum(y[train_idxs])/len(train_idxs),sum(y[test_idxs])/len(test_idxs))

===== GroupShuffleSplit =====
(2400, 32, 128) (2400,) (2400,)
(1800,) (600,) set()
[0.66666667] [0.4]
(1800,) (600,) set()
[0.56666667] [0.7]
(1800,) (600,) set()
[0.63333333] [0.5]
(1800,) (600,) set()
[0.66666667] [0.4]
(1800,) (600,) set()
[0.63333333] [0.5]
(1800,) (600,) set()
[0.6] [0.6]
(1800,) (600,) set()
[0.53333333] [0.8]
(1800,) (600,) set()
[0.7] [0.3]
(1800,) (600,) set()
[0.56666667] [0.7]
(1800,) (600,) set()
[0.56666667] [0.7]

===== StratifiedGroupKFold =====
(2400, 32, 128) (2400,) (2400,)
(2160,) (240,) set()
[0.61111111] [0.5]
(2160,) (240,) set()
[0.58333333] [0.75]
(2160,) (240,) set()
[0.55555556] [1.]
(2160,) (240,) set()
[0.61111111] [0.5]
(2160,) (240,) set()
[0.61111111] [0.5]
(2160,) (240,) set()
[0.58333333] [0.75]
(2160,) (240,) set()
[0.61111111] [0.5]
(2160,) (240,) set()
[0.61111111] [0.5]
(2160,) (240,) set()
[0.61111111] [0.5]
(2160,) (240,) set()
[0.61111111] [0.5]

===== StratifiedShuffleSplit =====
(2400, 32, 128) (2400,) (2400,)
(1800,) (600,) {0

As a result, I will use `GroupShuffleSplit`.

In [13]:
def train_model(X_ori,y_ori,groups_ori,filename=None, kernel='rbf'):
    
    # Make a copy because I am paranoid
    X,y,groups = X_ori.copy(), y_ori.copy(), groups_ori.copy()

    from sklearn.svm import SVC
    from sklearn.model_selection import GroupShuffleSplit 
    from sklearn.model_selection import cross_val_score

    model = SVC(kernel=kernel,max_iter=50000)
    cv = GroupShuffleSplit (n_splits=10, train_size=0.75, random_state=0)
    cross = cross_val_score(model, X, y, cv=cv,groups=groups, n_jobs=8)
    
    # We probably dont need this
    model = SVC(kernel=kernel, max_iter=50000)
    model.fit(X, y)
    ans = model.predict(X)
    acc = sum(ans == y) / len(y)
    # If the model answer with all 0 or 1, we print this message
    if( sum(ans) == len(y) or sum(ans) == 0 ): 
        print(f"-----WARNING: Model {filename} failed to learn: sum(ans)={sum(ans)} sum(y)={sum(y)} len(y)={len(y)}")
    return model, acc, cross

## 2. Spectral Features

In [14]:
from mne_features.feature_extraction import FeatureExtractor

In [15]:
bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})
fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)

accs,cv_means,cv_stds  = [],[],[]
# count = 0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    X = fe.fit_transform(X=data)
    _, acc, cross = train_model(X, labels.reshape(-1), groups,filename=filename)
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
    # count+=1
    # if(count == 5): break
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

accs_v,cv_means_v,cv_stds_v  = [],[],[]
# count=0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    X = fe.fit_transform(X=data)
    _, acc, cross = train_model(X, labels.reshape(-1), groups,filename=filename)
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # count+=1
    # if(count == 5): break
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.76917|10-CV=0.54533|STD=0.12671|Time spend=53.19175410270691
	AROUSAL-s02|Acc=0.64292|10-CV=0.54367|STD=0.07663|Time spend=4.825149774551392
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=480.0 len(y)=2400
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=3.9972658157348633
	AROUSAL-s04|Acc=0.65292|10-CV=0.52267|STD=0.09443|Time spend=4.7857325077056885
	AROUSAL-s05|Acc=0.63667|10-CV=0.423|STD=0.10057|Time spend=5.007667779922485
	AROUSAL-s06|Acc=0.63292|10-CV=0.5135|STD=0.12294|Time spend=4.671141862869263
	AROUSAL-s07|Acc=0.72833|10-CV=0.58867|STD=0.08981|Time spend=4.50341272354126
	AROUSAL-s08|Acc=0.72417|10-CV=0.42967|STD=0.11511|Time spend=4.5084474086761475
	AROUSAL-s09|Acc=0.67792|10-CV=0.48783|STD=0.07916|Time spend=4.522738933563232
	AROUSAL-s10|Acc=0.75958|10-CV=0.6025|STD=0.08021|Time spend=4.503004312515259
	AROUSAL-s11|Acc=0.62917|10-CV=0.579|STD=0.05024|Time spend=4.713841438293457
-----WARNING: Model s12 failed to learn: sum(ans)=24

## 3. Asymetry

In [16]:
channels = ['Fp1','AF3','F3','F7','FC5','FC1','C3','T7','CP5','CP1','P3','P7','PO3','O1','Oz','Pz','Fp2','AF4','Fz','F4','F8','FC6','FC2','Cz','C4','T8','CP6','CP2','P4','P8','PO4','O2']
left_channels = ['Fp1','F7','F3','T7','P7','C3','P3','O1','AF3','FC5','FC1','CP5','CP1','PO3']
right_channels = ['Fp2','F8','F4','T8','P8','C4','P4','O2','AF4','FC6','FC2','CP6','CP2','PO4']
left_channel_indexes = [ channels.index(ch) for ch in left_channels ]
right_channel_indexes = [ channels.index(ch) for ch in right_channels ]

print(f"{left_channel_indexes=}")
print(f"{right_channel_indexes=}")

frontal_channels = ['FC5','FC1','FC2','FC6','F7','F3','Fz','F4','F8','Fp1','Fp2']
posterior_channels = ['CP5','CP1','CP2','CP6','P7','P3','Pz','P4','P8','O1','O2']

frontal_channel_indexes = [ channels.index(ch) for ch in frontal_channels ]
posterior_channel_indexes = [ channels.index(ch) for ch in posterior_channels ]

print(f"{frontal_channel_indexes=}")
print(f"{posterior_channel_indexes=}")



left_channel_indexes=[0, 3, 2, 7, 11, 6, 10, 13, 1, 4, 5, 8, 9, 12]
right_channel_indexes=[16, 20, 19, 25, 29, 24, 28, 31, 17, 21, 22, 26, 27, 30]
frontal_channel_indexes=[4, 5, 22, 21, 3, 2, 18, 19, 20, 0, 16]
posterior_channel_indexes=[8, 9, 27, 26, 11, 10, 15, 28, 29, 13, 31]


In [17]:
bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})

accs,cv_means,cv_stds  = {'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]}
# count = 0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)

    PSD_left = X[:, left_channel_indexes].copy()
    PSD_right = X[:, right_channel_indexes].copy()
    PSD_frontal = X[:, frontal_channel_indexes].copy()
    PSD_posterior = X[:, posterior_channel_indexes].copy()
    X = {
        'DASM': PSD_left - PSD_right,
        'RASM': PSD_left / PSD_right,
        'DCAU': PSD_frontal - PSD_posterior,
    }

    for feature_name in ['DASM','RASM','DCAU']:
        _, acc, cross = train_model(X[feature_name], labels.reshape(-1), groups, filename=filename)
        print(f"\tAROUSAL-{filename}-{feature_name}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
        accs[feature_name].append(acc)
        cv_means[feature_name].append(cross.mean())
        cv_stds[feature_name].append(cross.std())
    # count+=1
    # if(count == 3): break
for feature_name in ['DASM','RASM','DCAU']:
    print(f"AROUSAL-{feature_name}|Acc={sum(accs[feature_name])/len(accs[feature_name])}|10-CV={sum(cv_means[feature_name])/len(cv_means[feature_name])}|STD={sum(cv_stds[feature_name])/len(cv_stds[feature_name])}")

accs_v,cv_means_v,cv_stds_v  = {'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]}
# count=0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)

    PSD_left = X[:, left_channel_indexes].copy()
    PSD_right = X[:, right_channel_indexes].copy()
    PSD_frontal = X[:, frontal_channel_indexes].copy()
    PSD_posterior = X[:, posterior_channel_indexes].copy()
    X = {
        'DASM': PSD_left - PSD_right,
        'RASM': PSD_left / PSD_right,
        'DCAU': PSD_frontal - PSD_posterior,
    }

    for feature_name in ['DASM','RASM','DCAU']:
        _, acc, cross = train_model(X[feature_name], labels.reshape(-1), groups, filename=filename)
        print(f"\tVALENCE-{filename}-{feature_name}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
        accs_v[feature_name].append(acc)
        cv_means_v[feature_name].append(cross.mean())
        cv_stds_v[feature_name].append(cross.std())
    # count+=1
    # if(count == 3): break
for feature_name in ['DASM','RASM','DCAU']:
    print(f"VALENCE-{feature_name}|Acc={sum(accs_v[feature_name])/len(accs_v[feature_name])}|10-CV={sum(cv_means_v[feature_name])/len(cv_means_v[feature_name])}|STD={sum(cv_stds_v[feature_name])/len(cv_stds_v[feature_name])}")

for feature_name in ['DASM','RASM','DCAU']:
    accs[feature_name].extend(accs_v[feature_name])
    cv_means[feature_name].extend(cv_means_v[feature_name])
    cv_stds[feature_name].extend(cv_stds_v[feature_name])
    print(f"TOTAL-{feature_name}|Acc={sum(accs[feature_name])/len(accs[feature_name])}|10-CV={sum(cv_means[feature_name])/len(cv_means[feature_name])}|STD={sum(cv_stds[feature_name])/len(cv_stds[feature_name])}")

	AROUSAL-s01-DASM|Acc=0.61|10-CV=0.52683|STD=0.12269|Time spend=2.5244381427764893
	AROUSAL-s01-RASM|Acc=0.60833|10-CV=0.55783|STD=0.15148|Time spend=3.6488218307495117
	AROUSAL-s01-DCAU|Acc=0.63167|10-CV=0.529|STD=0.1247|Time spend=4.637875318527222
	AROUSAL-s02-DASM|Acc=0.615|10-CV=0.52283|STD=0.07245|Time spend=2.2942090034484863
	AROUSAL-s02-RASM|Acc=0.60708|10-CV=0.57783|STD=0.14908|Time spend=3.4416568279266357
	AROUSAL-s02-DCAU|Acc=0.61583|10-CV=0.52333|STD=0.0682|Time spend=4.432082414627075
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=480.0 len(y)=2400
	AROUSAL-s03-DASM|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=2.0978734493255615
	AROUSAL-s03-RASM|Acc=0.80208|10-CV=0.72933|STD=0.15494|Time spend=2.7306392192840576
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=480.0 len(y)=2400
	AROUSAL-s03-DCAU|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=3.656728744506836
	AROUSAL-s04-DASM|Acc=0.63333|10-CV=0.54267|STD=0.10763|Time spend=3.041966676712036
	AROUSAL-

## 4. Connectivity

### 4.1 coherence

In [18]:
from mne_connectivity import spectral_connectivity_time

def save(data,filename):
    with open(f'cache/{filename}.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load(filename):
    with open(f'cache/{filename}.pickle', 'rb') as handle:
        data = pickle.load(handle)
    return data

In [4]:
# bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
for filename in tqdm(dataset.get_file_list()):
    # if(filename in ['s01','s02','s03','s04','s05','s06','s07']): continue
    print("----"*5, filename, "----"*5)
    # start = time.time()
    epochs, _ = dataset.get_data_mne_epochs(filename, sfreq=128, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, split=False)
    # X = fe.fit_transform(X=data)
    # print(epochs, labels.shape)
    step = 150//2
    X = []
    # count = 0
    for index in range(0,len(epochs), step):
        # print(index,index+step)
        conn = spectral_connectivity_time(epochs[index:index+step], method='coh',mode='multitaper',sfreq=128, freqs=range(7,64), verbose='CRITICAL')
        # (step, 528, 57, 128) => (epoch, conn, freq, time) .mean(axis=3)
        # (step, 528, 57) => (epoch, conn, freq)
        # print(conn.get_data('raveled').mean(axis=3).shape)
        conn = conn.get_data('raveled').mean(axis=3)
        # (step, 528*4 ) =>(step, 2112 )
        x = np.concatenate([ conn[:,:,7-7:8-7 ].mean(axis=2),
                            conn[:,:,8-7:12-7 ].mean(axis=2),
                            conn[:,:,12-7:30-7 ].mean(axis=2),
                            conn[:,:,30-7:64-7 ].mean(axis=2) ],axis=1)
        X.append(x)
        # count += 1
        # if(count == 3): break
    X = np.vstack(X)
    print(X.shape)
    save(X,f"X_COH_AROUSAL_{filename}")


    epochs, _ = dataset.get_data_mne_epochs(filename, sfreq=128, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, split=False)
    # X = fe.fit_transform(X=data)
    # print(epochs, labels.shape)
    step = 150//2
    X = []
    # count = 0
    for index in tqdm(range(0,len(epochs), step)):
        # print(index,index+step)
        conn = spectral_connectivity_time(epochs[index:index+step], method='coh',mode='multitaper',sfreq=128, freqs=range(7,64), verbose='CRITICAL')
        # (step, 528, 57, 128) => (epoc
        # h, conn, freq, time) .mean(axis=3)
        # (step, 528, 57) => (epoch, conn, freq)
        # print(conn.get_data('raveled').mean(axis=3).shape)
        conn = conn.get_data('raveled').mean(axis=3)
        # (step, 528*4 ) =>(step, 2112 )
        x = np.concatenate([ conn[:,:,7-7:8-7 ].mean(axis=2),
                            conn[:,:,8-7:12-7 ].mean(axis=2),
                            conn[:,:,12-7:30-7 ].mean(axis=2),
                            conn[:,:,30-7:64-7 ].mean(axis=2) ],axis=1)
        X.append(x)
        # count += 1
        # if(count == 3): break
    X = np.vstack(X)
    print(X.shape)
    save(X,f"X_COH_VALENCE_{filename}")

In [22]:
accs, cv_means, cv_stds = [],[],[]
for filename in tqdm(dataset.get_file_list()):
    start = time.time()
    X = load(f"X_COH_AROUSAL_{filename}")
    assert X.shape == (2400,2112)
    _, labels,groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    _, acc, cross = train_model(X, labels.reshape(-1), groups,filename=filename)
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

	AROUSAL-s01|Acc=0.77917|10-CV=0.54617|STD=0.14184|Time spend=72.09873604774475
	AROUSAL-s02|Acc=0.70292|10-CV=0.54167|STD=0.07741|Time spend=53.257426500320435
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=480.0 len(y)=2400
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=36.00350046157837
	AROUSAL-s04|Acc=0.73042|10-CV=0.53633|STD=0.10344|Time spend=49.57472538948059
	AROUSAL-s05|Acc=0.72083|10-CV=0.47983|STD=0.10048|Time spend=57.4552948474884
	AROUSAL-s06|Acc=0.67792|10-CV=0.5205|STD=0.12185|Time spend=57.38243746757507
	AROUSAL-s07|Acc=0.76|10-CV=0.57767|STD=0.0818|Time spend=54.6458694934845
	AROUSAL-s08|Acc=0.76417|10-CV=0.43267|STD=0.11995|Time spend=54.8598849773407
	AROUSAL-s09|Acc=0.75417|10-CV=0.4935|STD=0.07972|Time spend=51.59825849533081
	AROUSAL-s10|Acc=0.81292|10-CV=0.6625|STD=0.08202|Time spend=50.832497358322144
	AROUSAL-s11|Acc=0.67542|10-CV=0.552|STD=0.04605|Time spend=51.17717623710632
-----WARNING: Model s12 failed to learn: sum(ans)=2400.0 s

In [24]:
accs_v, cv_means_v, cv_stds_v = [],[],[]
for filename in tqdm(dataset.get_file_list()):
    start = time.time()
    X = load(f"X_COH_VALENCE_{filename}")
    assert X.shape == (2400,2112)
    _, labels,groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    _, acc, cross = train_model(X, labels.reshape(-1), groups,filename=filename)
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

  0%|          | 0/32 [00:00<?, ?it/s]

	VALENCE-s01|Acc=0.72|10-CV=0.41933|STD=0.06797|Time spend=59.25682973861694
	VALENCE-s02|Acc=0.7475|10-CV=0.65983|STD=0.10715|Time spend=47.22028207778931
	VALENCE-s03|Acc=0.75333|10-CV=0.56583|STD=0.16636|Time spend=53.59569239616394
	VALENCE-s04|Acc=0.77125|10-CV=0.60717|STD=0.07833|Time spend=48.5079128742218
	VALENCE-s05|Acc=0.71875|10-CV=0.56567|STD=0.0848|Time spend=53.26034903526306
-----WARNING: Model s06 failed to learn: sum(ans)=2400.0 sum(y)=1800.0 len(y)=2400
	VALENCE-s06|Acc=0.75|10-CV=0.71767|STD=0.0736|Time spend=49.34060454368591
	VALENCE-s07|Acc=0.76583|10-CV=0.6585|STD=0.13381|Time spend=46.04002046585083
	VALENCE-s08|Acc=0.74125|10-CV=0.48433|STD=0.08415|Time spend=58.035321950912476
	VALENCE-s09|Acc=0.78542|10-CV=0.64483|STD=0.0799|Time spend=47.633655309677124
	VALENCE-s10|Acc=0.85083|10-CV=0.78033|STD=0.08143|Time spend=38.175557374954224
	VALENCE-s11|Acc=0.68708|10-CV=0.50083|STD=0.13131|Time spend=52.31644153594971
	VALENCE-s12|Acc=0.75958|10-CV=0.54117|STD=0.0

In [25]:
accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

TOTAL|Acc=0.7467968749999996|10-CV=0.5999661458333331|STD=0.09557871591333594


### Connectivity PLV

In [ ]:
# bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
for filename in tqdm(dataset.get_file_list()):
    # if(filename in ['s01','s02','s03','s04','s05','s06','s07']): continue
    print("----"*5, filename, "----"*5)
    # start = time.time()
    epochs, _ = dataset.get_data_mne_epochs(filename, sfreq=128, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, split=False)
    # X = fe.fit_transform(X=data)
    # print(epochs, labels.shape)
    step = 150//2
    X = []
    # count = 0
    for index in range(0,len(epochs), step):
        # print(index,index+step)
        conn = spectral_connectivity_time(epochs[index:index+step], method='plv',mode='multitaper',sfreq=128, freqs=range(7,64), verbose='CRITICAL')
        # (step, 528, 57, 128) => (epoch, conn, freq, time) .mean(axis=3)
        # (step, 528, 57) => (epoch, conn, freq)
        # print(conn.get_data('raveled').mean(axis=3).shape)
        conn = conn.get_data('raveled').mean(axis=3)
        # (step, 528*4 ) =>(step, 2112 )
        x = np.concatenate([ conn[:,:,7-7:8-7 ].mean(axis=2),
                            conn[:,:,8-7:12-7 ].mean(axis=2),
                            conn[:,:,12-7:30-7 ].mean(axis=2),
                            conn[:,:,30-7:64-7 ].mean(axis=2) ],axis=1)
        X.append(x)
        # count += 1
        # if(count == 3): break
    X = np.vstack(X)
    print(X.shape)
    save(X,f"X_PLV_AROUSAL_{filename}")


    epochs, _ = dataset.get_data_mne_epochs(filename, sfreq=128, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, split=False)
    # X = fe.fit_transform(X=data)
    # print(epochs, labels.shape)
    step = 150//2
    X = []
    # count = 0
    for index in tqdm(range(0,len(epochs), step)):
        # print(index,index+step)
        conn = spectral_connectivity_time(epochs[index:index+step], method='plv',mode='multitaper',sfreq=128, freqs=range(7,64), verbose='CRITICAL')
        # (step, 528, 57, 128) => (epoc
        # h, conn, freq, time) .mean(axis=3)
        # (step, 528, 57) => (epoch, conn, freq)
        # print(conn.get_data('raveled').mean(axis=3).shape)
        conn = conn.get_data('raveled').mean(axis=3)
        # (step, 528*4 ) =>(step, 2112 )
        x = np.concatenate([ conn[:,:,7-7:8-7 ].mean(axis=2),
                            conn[:,:,8-7:12-7 ].mean(axis=2),
                            conn[:,:,12-7:30-7 ].mean(axis=2),
                            conn[:,:,30-7:64-7 ].mean(axis=2) ],axis=1)
        X.append(x)
        # count += 1
        # if(count == 3): break
    X = np.vstack(X)
    print(X.shape)
    save(X,f"X_PLV_VALENCE_{filename}")

In [26]:
accs, cv_means, cv_stds = [],[],[]
for filename in tqdm(dataset.get_file_list()):
    start = time.time()
    X = load(f"X_PLV_AROUSAL_{filename}")
    assert X.shape == (2400,2112)
    _, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    _, acc, cross = train_model(X, labels.reshape(-1), groups, filename=filename)
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

	AROUSAL-s01|Acc=0.75542|10-CV=0.54167|STD=0.13722|Time spend=58.2156548500061
	AROUSAL-s02|Acc=0.71083|10-CV=0.54533|STD=0.07539|Time spend=51.67206573486328
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=480.0 len(y)=2400
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=37.107110261917114
	AROUSAL-s04|Acc=0.7325|10-CV=0.53583|STD=0.10332|Time spend=53.449044942855835
	AROUSAL-s05|Acc=0.72875|10-CV=0.48717|STD=0.09655|Time spend=60.510722637176514
	AROUSAL-s06|Acc=0.67542|10-CV=0.519|STD=0.12047|Time spend=48.775301933288574
	AROUSAL-s07|Acc=0.75375|10-CV=0.5785|STD=0.08497|Time spend=47.53772449493408
	AROUSAL-s08|Acc=0.75625|10-CV=0.4355|STD=0.12245|Time spend=49.33205270767212
	AROUSAL-s09|Acc=0.75542|10-CV=0.4955|STD=0.07933|Time spend=46.42083978652954
	AROUSAL-s10|Acc=0.80958|10-CV=0.65917|STD=0.07514|Time spend=50.13776922225952
	AROUSAL-s11|Acc=0.6925|10-CV=0.55567|STD=0.05022|Time spend=50.09951138496399
-----WARNING: Model s12 failed to learn: sum(ans)=24

In [27]:
accs_v, cv_means_v, cv_stds_v = [],[],[]
for filename in tqdm(dataset.get_file_list()):
    start = time.time()
    X = load(f"X_PLV_VALENCE_{filename}")
    assert X.shape == (2400,2112)
    _, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    _, acc, cross = train_model(X, labels.reshape(-1), groups, filename=filename)
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)

print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

	VALENCE-s01|Acc=0.71708|10-CV=0.42483|STD=0.06913|Time spend=58.38158130645752
	VALENCE-s02|Acc=0.75208|10-CV=0.654|STD=0.10568|Time spend=45.17832064628601
	VALENCE-s03|Acc=0.7575|10-CV=0.56317|STD=0.16687|Time spend=53.06384229660034
	VALENCE-s04|Acc=0.77292|10-CV=0.59483|STD=0.07703|Time spend=46.40867495536804
	VALENCE-s05|Acc=0.72583|10-CV=0.56583|STD=0.08497|Time spend=51.74175024032593
-----WARNING: Model s06 failed to learn: sum(ans)=2400.0 sum(y)=1800.0 len(y)=2400
	VALENCE-s06|Acc=0.75|10-CV=0.71867|STD=0.07457|Time spend=45.86181974411011
	VALENCE-s07|Acc=0.75958|10-CV=0.66133|STD=0.13482|Time spend=42.283950090408325
	VALENCE-s08|Acc=0.75125|10-CV=0.48317|STD=0.08322|Time spend=54.72673439979553
	VALENCE-s09|Acc=0.79333|10-CV=0.6375|STD=0.0813|Time spend=45.475549936294556
	VALENCE-s10|Acc=0.84917|10-CV=0.77767|STD=0.08128|Time spend=36.27791213989258
	VALENCE-s11|Acc=0.70208|10-CV=0.507|STD=0.1285|Time spend=49.72342801094055
	VALENCE-s12|Acc=0.76083|10-CV=0.5475|STD=0.09

## 5. CSP

In [28]:
from components.CSP import CSP
from mne_features.feature_extraction import FeatureExtractor

bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})

In [29]:
accs, cv_means, cv_stds = [],[],[]
for filename in tqdm(dataset.get_file_list()):

    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    labels = labels.squeeze()
    start = time.time()
    filters = CSP(data[labels==0], data[labels==1])
    
    csp_data = []
    for index in range(len(data)):
        
        # (672, 32) @ (32, 32) = (672, 32)
        csp = data[index].T @ filters[labels[index].astype(int)]
        # (1, 32, 672)
        csp = np.expand_dims(csp.T, axis=0)
        csp_data.append(csp)

    csp_data = np.vstack(csp_data)
    # print(csp_data.shape) => (2400, 32, 128)
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=csp_data)
    # print(X.shape, time.time() - start) => (2400, 160) 1.2374355792999268

    assert X.shape == (2400,160)
    _, acc, cross = train_model(X, labels.reshape(-1), groups, filename=filename)
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


-----WARNING: Model s01 failed to learn: sum(ans)=2400.0 sum(y)=1440.0 len(y)=2400
	AROUSAL-s01|Acc=0.6|10-CV=0.53883|STD=0.13232|Time spend=33.491339683532715


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s02|Acc=0.80667|10-CV=0.7825|STD=0.04279|Time spend=3.3040568828582764


...

	AROUSAL-s30|Acc=0.83833|10-CV=0.821|STD=0.0538|Time spend=2.9493160247802734


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s31|Acc=0.94958|10-CV=0.94283|STD=0.01138|Time spend=2.023477792739868


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s32|Acc=0.90542|10-CV=0.8865|STD=0.05254|Time spend=2.436899185180664
AROUSAL|Acc=0.7999088541666667|10-CV=0.7664531250000001|STD=0.06639725666649411


In [30]:
accs_v, cv_means_v, cv_stds_v = [],[],[]
for filename in tqdm(dataset.get_file_list()):

    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    labels = labels.squeeze()
    start = time.time()
    filters = CSP(data[labels==0], data[labels==1])
    
    csp_data = []
    for index in range(len(data)):
        
        # (672, 32) @ (32, 32) = (672, 32)
        csp = data[index].T @ filters[labels[index].astype(int)]
        # (1, 32, 672)
        csp = np.expand_dims(csp.T, axis=0)
        csp_data.append(csp)

    csp_data = np.vstack(csp_data)
    # print(csp_data.shape) => (2400, 32, 128)
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=csp_data)
    # print(X.shape, time.time() - start) => (2400, 160) 1.2374355792999268

    assert X.shape == (2400,160)
    _, acc, cross = train_model(X, labels.reshape(-1), groups, filename=filename)
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

  0%|          | 0/32 [00:00<?, ?it/s]

/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s01|Acc=0.55375|10-CV=0.42917|STD=0.06762|Time spend=4.513397693634033


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s02|Acc=0.99667|10-CV=0.99567|STD=0.00318|Time spend=1.6004712581634521


...

	VALENCE-s30|Acc=0.85|10-CV=0.8265|STD=0.04829|Time spend=2.9235219955444336


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s31|Acc=0.58833|10-CV=0.48933|STD=0.10852|Time spend=4.774028301239014


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s32|Acc=0.57417|10-CV=0.45967|STD=0.13675|Time spend=4.643377065658569
VALENCE|Acc=0.7356640625|10-CV=0.6738489583333334|STD=0.08137207163881335


In [31]:
accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)

print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

TOTAL|Acc=0.7677864583333331|10-CV=0.7201510416666669|STD=0.07388466415265374
